In [ ]:
!pip install gurobipy yfinance
!pip install cvxpy
!pip install pulp

import pandas as pd
import numpy as np
import yfinance as yf
from cvxpy import *
import seaborn as sns
import matplotlib.pyplot as plt
from pulp import *

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import requests 
from bs4 import BeautifulSoup 

In [ ]:
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

## DATA COLLECTION

In [ ]:
wikiurl="https://en.wikipedia.org/wiki/List_of_S%26P_600_companies"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)

soup = BeautifulSoup(response.text, 'html.parser')
small_cap=soup.find('table',{'class':"wikitable"})

In [ ]:
df = pd.read_html(str(small_cap))
df = pd.DataFrame(df[0])
df.head()

,Company,Ticker symbol,GICS Sector,GICS Sub-Industry,SEC filings,CIK
0,The Aaron's Company,AAN,Consumer Discretionary,Homefurnishing Retail,view,1821393
1,"AAON, Inc.",AAON,Industrials,Building Products,view,824142
2,American Assets Trust,AAT,Real Estate,Diversified REITs,view,1500217
3,Atlas Air Worldwide Holdings,AAWW,Industrials,Air Freight & Logistics,view,1135185
4,Ameris Bancorp,ABCB,Financials,Regional Banks,view,351569


In [ ]:
df = df[['Ticker symbol', 'GICS Sector']]

In [ ]:
df['GICS Sector'].value_counts()

Financials                106
Consumer Discretionary     91
Industrials                88
Health Care                81
Information Technology     63
Real Estate                55
Materials                  39
Energy                     32
Consumer Staples           28
Communication Services     10
Utilities                   8
Name: GICS Sector, dtype: int64

In [ ]:
finance_small = list(df['Ticker symbol'][df['GICS Sector'] == 'Financials'])
healthcare_small = list(df['Ticker symbol'][df['GICS Sector'] == 'Health Care'])
technology_small = list(df['Ticker symbol'][df['GICS Sector'] == 'Information Technology'])
real_estate_small = list(df['Ticker symbol'][df['GICS Sector'] == 'Real Estate'])
industrial_small = list(df['Ticker symbol'][df['GICS Sector'] == 'Industrials'])

small = finance_small + healthcare_small + technology_small + real_estate_small + industrial_small

In [ ]:
len(small)

393

In [ ]:
small_cap = yf.download(small, end = '2021-06-01', start = '2016-01-01', interval='3mo')

[*********************100%***********************]  393 of 393 completed

4 Failed downloads:
- ONL: Data doesn't exist for startDate = 1451606400, endDate = 1622505600
- MOG.A: No data found, symbol may be delisted
- FBRT: Data doesn't exist for startDate = 1451606400, endDate = 1622505600
- CCSI: Data doesn't exist for startDate = 1451606400, endDate = 1622505600


In [ ]:
small_cap.Close.shape

(1181, 393)

In [ ]:
small_cap.Close.head()

,AAON,AAT,AAWW,ABCB,ABM,ABTX,ACA,ACLS,ADC,ADTN,ADUS,AEIS,AEL,AGO,AGYS,AHH,AIN,AIR,AIT,AJRD,AKR,ALEX,ALG,ALGT,ALRM,AMBC,AMEH,AMN,AMPH,AMSF,AMWD,ANGO,ANIK,ANIP,APOG,ARCB,ARI,ARLO,ARR,ASTE,AVAV,AVNS,AX,AZZ,B,BANC,BANF,BANR,BCC,BCOR,BDN,BFS,BHE,BHLB,BKU,BLFS,BMI,BRC,BRKL,BSIG,CAMP,CARA,CASH,CBU,CCRN,CCSI,CDMO,CEVA,CFFN,CHCO,CHCT,CHRS,CIR,CLDT,CLI,CMTL,CNMD,CNXN,COHU,COLB,COLL,COOP,CORT,CPF,CPSI,CRVL,CRY,CSGS,CSII,CSR,CTRE,CTS,CUBI,CUTR,CVBF,CVET,CXW,CYH,CYTK,DBD,DCOM,DDD,DEA,DFIN,DGII,DHC,DIOD,DLX,DNOW,DRH,DSPG,DXPE,EBIX,ECOL,ECPG,EFC,EGBN,EGHT,EGRX,EHTH,EIG,ENDP,ENSG,ENTA,ENVA,EPAC,EPAY,EPRT,ESE,EVTC,EXLS,EXPO,EXTR,EZPW,FARO,FBC,FBK,FBNC,FBP,FBRT,FCF,FCPT,FELE,FFBC,FHB,FIX,FLGT,FLOW,FMBI,FN,FORM,FORR,FSP,FSS,FWRD,GBX,GDOT,GEO,GFF,GHL,GKOS,GMS,GNL,GNW,GPMT,GTLS,GTY,GVA,GWB,HA,HAFC,HCI,HCSG,HFWA,HI,HLIT,HMN,HMST,HNGR,HNI,HOPE,HRMY,HSC,HSII,HSKA,HSTM,HT,HTH,HTLD,HUBG,IBTX,ICHR,IDCC,IIIN,IIPR,ILPT,INDB,INGN,INN,INVA,IRT,ISBC,ITGR,ITRI,IVR,JBT,JOE,JRVR,JYNT,KAMN,KELYA,KFY,KLIC,KN,KREF,LGND,LKFN,LMAT,LNTH,LPSN,LTC,LXP,MANT,MATW,MATX,MD,MDRX,MEI,MGLN,MLAB,MLI,MMI,MMSI,MODV,MOG.A,MRTN,MTOR,MTRX,MXL,MYGN,MYRG,NBHC,NBTB,NEO,NFBK,NMIH,NPK,NPO,NSIT,NTCT,NTGR,NTUS,NWBI,NX,NXGN,NXRT,NYMT,OFG,OFIX,OMCL,OMI,ONB,ONL,ONTO,OPI,OPRX,ORGO,OSIS,OSPN,OSUR,PAHC,PBH,PBI,PCRX,PDFS,PFBC,PFS,PGTI,PIPR,PKE,PLAB,PLMR,PLUS,PLXS,PMT,PNTG,POLY,POWI,POWL,PPBI,PRA,PRAA,PRFT,PRGS,PRK,PRLB,QURE,RAVN,RC,RDNT,REZI,RGNX,RGP,RILY,RLGY,RMAX,RMBS,RNST,ROCK,ROG,ROIC,RPT,RWT,SAFE,SAFT,SANM,SBCF,SBSI,SCSC,SEM,SFBS,SFNC,SGH,SITC,SKT,SKYW,SLP,SLQT,SNEX,SPNT,SPPI,SPSC,SPXC,SRDX,STAR,STBA,STC,SUPN,SVC,SXI,TBBK,TBI,TBK,TCMD,TGI,TILE,TMP,TNC,TREE,TRHC,TRMK,TRST,TRUP,TTEC,TTMI,TVTY,TWO,UBA,UCBI,UCTT,UFCS,UFPI,UHT,UIS,UNF,UNIT,USPH,UVE,VBTX,VCEL,VECO,VIAV,VICR,VIVO,VNDA,VREX,VRTS,VRTV,VVI,WABC,WD,WETF,WIRE,WNC,WRE,WRLD,WSFS,WSR,WTS,XHR,XNCR,XPER,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01,28.0,39.919998,42.27,29.58,32.310001,18.370001,NaN,11.2,38.470001,20.219999,17.190001,34.790001,16.799999,25.299999,10.21,11.25,37.59,23.27,43.400002,16.379999,35.130001,36.68,55.709

In [ ]:
small_cap = small_cap.Close

In [ ]:
small_drop = list(small_cap.iloc[1,:][small_cap.iloc[1,:].isnull()].index)

In [ ]:
small_cap.drop(small_drop, axis = 1, inplace = True)

In [ ]:
small_cap.dropna(axis = 0, inplace = True)

In [ ]:
small = small_cap.columns

In [ ]:
len(small)

363

In [ ]:
for stock in finance_small:
  if stock not in small:
    finance_small.remove(stock)

for stock in healthcare_small:
  if stock not in small:
    healthcare_small.remove(stock)

for stock in technology_small:
  if stock not in small:
    technology_small.remove(stock)

for stock in real_estate_small:
  if stock not in small:
    real_estate_small.remove(stock)
    
for stock in industrial_small:
  if stock not in small:
    industrial_small.remove(stock)

In [ ]:
small = list(small_cap.columns)

In [ ]:
wikiurl="https://en.wikipedia.org/wiki/List_of_S%26P_400_companies"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)

soup = BeautifulSoup(response.text, 'html.parser')
mid_cap=soup.find('table',{'class':"wikitable"})

In [ ]:
df = pd.read_html(str(mid_cap))
df = pd.DataFrame(df[0])
df.head()

,Security,Ticker symbol,GICS Sector,GICS Sub-Industry,SEC filings
0,American Campus Communities,ACC,Real Estate,Residential REITs,reports
1,Acadia Healthcare,ACHC,Health Care,Health Care Facilities,reports
2,ACI Worldwide,ACIW,Information Technology,Application Software,reports
3,AECOM,ACM,Industrials,Construction & Engineering,reports
4,Alliance Data Systems,ADS,Information Technology,Data Processing & Outsourced Services,reports


In [ ]:
finance_mid = list(df['Ticker symbol'][df['GICS Sector'] == 'Financials'])
healthcare_mid = list(df['Ticker symbol'][df['GICS Sector'] == 'Health Care'])
technology_mid = list(df['Ticker symbol'][df['GICS Sector'] == 'Information Technology'])
real_estate_mid = list(df['Ticker symbol'][df['GICS Sector'] == 'Real Estate'])
industrial_mid = list(df['Ticker symbol'][df['GICS Sector'] == 'Industrials'])

mid = finance_mid + healthcare_mid + technology_mid + real_estate_mid + industrial_mid

In [ ]:
mid_cap = yf.download(mid, end = '2021-06-01', start = '2016-01-01', interval='3mo')

[*********************100%***********************]  261 of 261 completed

2 Failed downloads:
- GXO: Data doesn't exist for startDate = 1451606400, endDate = 1622505600
- KD: Data doesn't exist for startDate = 1451606400, endDate = 1622505600


In [ ]:
mid_cap = mid_cap.Close
mid_drop = list(mid_cap.iloc[0,:][mid_cap.iloc[0,:].isnull()].index)
mid_cap.drop(mid_drop, axis = 1, inplace = True)

In [ ]:
mid_cap.dropna(axis = 0, inplace = True)

In [ ]:
mid = mid_cap.columns

In [ ]:
for stock in finance_mid:
  if stock not in mid:
    finance_mid.remove(stock)

for stock in healthcare_mid:
  if stock not in mid:
    healthcare_mid.remove(stock)

for stock in technology_mid:
  if stock not in mid:
    technology_mid.remove(stock)

for stock in real_estate_mid:
  if stock not in mid:
    real_estate_mid.remove(stock)
    
for stock in industrial_mid:
  if stock not in mid:
    industrial_mid.remove(stock)

In [ ]:
wikiurl="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)

soup = BeautifulSoup(response.text, 'html.parser')
large_cap=soup.find('table',{'class':"wikitable"})

In [ ]:
df = pd.read_html(str(large_cap))
df = pd.DataFrame(df[0])
df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [ ]:
finance_large = list(df['Symbol'][df['GICS Sector'] == 'Financials'])
healthcare_large = list(df['Symbol'][df['GICS Sector'] == 'Health Care'])
technology_large = list(df['Symbol'][df['GICS Sector'] == 'Information Technology'])
real_estate_large = list(df['Symbol'][df['GICS Sector'] == 'Real Estate'])
industrial_large = list(df['Symbol'][df['GICS Sector'] == 'Industrials'])

large = finance_large + healthcare_large + technology_large + real_estate_large + industrial_large

In [ ]:
large_cap = yf.download(large, end = '2021-06-01', start = '2016-01-01', interval='3mo')

[*********************100%***********************]  306 of 306 completed

2 Failed downloads:
- OGN: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [ ]:
large_cap = large_cap.Close

In [ ]:
large_drop = list(large_cap.iloc[1,:][large_cap.iloc[1,:].isnull()].index)
large_cap.drop(large_drop, axis = 1, inplace = True)
large_cap.dropna(axis = 0, inplace = True)

In [ ]:
large = list(large_cap.columns)

In [ ]:
for stock in finance_large:
  if stock not in large:
    finance_large.remove(stock)

for stock in healthcare_large:
  if stock not in large:
    healthcare_large.remove(stock)

for stock in technology_large:
  if stock not in large:
    technology_large.remove(stock)

for stock in real_estate_large:
  if stock not in large:
    real_estate_large.remove(stock)
    
for stock in industrial_large:
  if stock not in large:
    industrial_large.remove(stock)

In [ ]:
large_cap.shape

(22, 297)

In [ ]:
small_cap.shape

(22, 363)

In [ ]:
mid_cap.shape

(22, 243)

In [ ]:
stocks = pd.concat([small_cap, mid_cap, large_cap], axis = 1)
stocks.shape

(22, 903)

In [ ]:
finance = finance_small + finance_mid + finance_large
healthcare = healthcare_small + healthcare_mid + healthcare_large
technology = technology_small + technology_mid + technology_large 
real_estate = real_estate_small + real_estate_mid + real_estate_small
industrial = industrial_small + industrial_mid + industrial_large

In [ ]:
len(finance)

224

In [ ]:
len(healthcare)

169

In [ ]:
len(technology)

179

In [ ]:
len(real_estate)

132

In [ ]:
len(industrial)

219

## CALCULATING RETURNS

In [ ]:
stocks_return = pd.DataFrame()
for i in stocks.columns:
  stocks_return[i] = np.divide(np.diff(stocks[i]),stocks[i][:-1] )

In [ ]:
expected_return = stocks_return.mean(axis = 0)

In [ ]:
cov = stocks_return.cov()

In [ ]:
n_assets = 10
n_portfolios = 1000
mean_variance_pairs = []

np.random.seed(75)

for i in range(n_portfolios):
    assets = list(np.random.choice(list(stocks.columns), n_assets, replace=False))
    weights = list(np.random.rand(n_assets))
    w = []
    for i in weights:
      w.append(i/sum(weights))
    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += w[i] * expected_return.loc[assets[i]]
        for j in range(len(assets)):
            portfolio_E_Variance += w[i] * w[j] * cov.loc[assets[i], assets[j]]
            
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])

In [ ]:
import plotly.graph_objects as go

In [ ]:

#-- Plot the risk vs. return of randomly generated portfolios
#- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)
risk_free_rate=0 #-- Include risk free rate here for sharpe ratio

#-- Create Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, 
                         y=mean_variance_pairs[:,0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Quarterly Risk (Volatility)'),
                  yaxis=dict(title='Quarterly Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

## MARKOWITZ OPTIMIZATION 

In [ ]:
return_data = stocks_return.T

In [ ]:
r = np.asarray(np.mean(return_data, axis=1))
C = np.asmatrix(np.cov(return_data))

In [ ]:
symbols = stocks_return.columns

# Number of variables
n = len(symbols)

# The variables vector
x = Variable(n)

# The minimum return
req_return = 0.05

# The return
ret = r.T*x

# The risk in xT.Q.x format
risk = quad_form(x, C)

# The core problem definition with the Problem class from CVXPY
prob = Problem(Minimize(risk), [sum(x)==1, ret >= req_return, x>=0, x<=1])

In [ ]:
prob.solve()
print ("Exp ret = {}%".format(round(100*ret.value,2)))
print ("Expected risk    = {}%".format(100*risk.value**0.5))

Exp ret = 5.0%
Expected risk    = 0.5391646769984839%


In [ ]:
for s in range(len(symbols)):
  if x.value[s]>0.00000000000001:
    print(symbols[s], ' = ', round(x.value[s],6))

AMN  =  0.095786
COLL  =  0.004878
EHTH  =  0.03764
GNW  =  0.020567
HCI  =  0.028904
OMI  =  0.007641
XPER  =  0.056077
CIEN  =  0.00546
EBS  =  0.031661
QDEL  =  0.012449
TNDM  =  0.001993
AKAM  =  0.067526
CME  =  0.21852
CTXS  =  0.065471
DLR  =  0.031071
LLY  =  0.076135
REGN  =  0.060518
ROL  =  0.007849
SBAC  =  0.169856
